In [2]:
from langchain_aws import ChatBedrock
from bs4 import BeautifulSoup
import pandas as pd
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

llm = ChatBedrock(
    model_id="us.anthropic.claude-3-haiku-20240307-v1:0",
    region = "us-east-1",    
    model_kwargs=dict(temperature=0)
)



In [5]:
matrix_data = pd.read_csv('data/cdti/matrix_data/cdti_matrix_data.csv')
matrix_data.head()

,instrument,support_ambit,name,url
0,Ayuda parcialmente reembolsable,Investigación y desarrollo,Proyectos de I + D,https://www.cdti.es/node/115
1,Ayuda parcialmente reembolsable,Investigación y desarrollo,Actuación conjunta ISCIII-CDTI,https://www.cdti.es/node/180
2,Ayuda parcialmente reembolsable,Investigación y desarrollo,Proyectos de I+D de Transferencia Tecnológica ...,https://www.cdti.es/node/18478
3,Ayuda parcialmente reembolsable,Investigación y desarrollo,Proyectos de I+D Aeroespacial y Salud,https://www.cdti.es/node/19033
4,Ayuda parcialmente reembolsable,Innovación,Línea Directa de Expansión,https://www.cdti.es/node/191


In [8]:
aid_data = matrix_data.iloc[0]
aid_data

instrument       Ayuda parcialmente reembolsable
support_ambit         Investigación y desarrollo
name                          Proyectos de I + D
url                 https://www.cdti.es/node/115
Name: 0, dtype: object

In [ ]:
from extractor.core.base_extractor import BaseExtractor
from extractor.params.extraction_params import (
    CDTI_MAIN_DIR, 
)

class CDTIAidExtractor(BaseExtractor):

    def __init__(
        self, 
        instrument,
        support_ambit,
        name,
        url):

        super().__init__()

        self.instrument=instrument
        self.support_ambit=support_ambit
        self.name=name
        self.url=url

        self.persist_data_dir = self.persist_data_dir + "/" + CDTI_MAIN_DIR + "/" + self.get_name_parsed()


    def get_name_parsed(self):
        return self.name.replace(" ", "_").lower()
